# Getting, Cleaning, and Preprocessing Data
The `SL132001-data (11).csv` file represents the data for the week of `26/05/2020`, and the `SL132001-data (12).csv` file represents the data for the week of `02/06/2020`. I concatenated these two datasets.  

In [41]:
# Import Libraries

import pandas as pd
import numpy as np
import datetime as dt

%matplotlib inline

In [42]:
# Reading 'SL132001-data (11).csv'
df_week1 = pd.read_csv('Data/SL132001-data (11).csv', skiprows=[1])

# Reading 'SL132001-data (12).csv'
df_week2 = pd.read_csv('Data/SL132001-data (12).csv', skiprows=[1])

In [43]:
# Concatenate the datasets

frames = [df_week1, df_week2]
df_complete = pd.concat(frames)

In [44]:
df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19744 entries, 0 to 9909
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Date                          19744 non-null  object 
 1   Time GMT -4                   19744 non-null  object 
 2   Timestamp                     19744 non-null  int64  
 3   Ozone - Low Conc.             19744 non-null  float64
 4   Hydrogen Sulfide - Low Conc.  19744 non-null  float64
 5   Total VOCs (ppm) - PID        19744 non-null  float64
 6   Carbon Dioxide - Low Conc.    19744 non-null  float64
 7   Particulate Matter 1          19744 non-null  float64
 8   Particulate Matter 2.5        19744 non-null  float64
 9   Particulate Matter 10         19744 non-null  float64
 10  Temperature (Internal)        19744 non-null  float64
 11  Humidity (Internal)           19744 non-null  float64
 12  Temperature (External)        19744 non-null  float64
 13  Hu

In [45]:
len(df_complete['Latitude'].unique()) == len(df_complete['Longitude'].unique()) == 1 

True

As seen above, 
- The `Unamed: 16` column is all null, and we need to drop it. 
- The `Latitude` and `Longtitude` columns are unique. Therefore, we need to drop these columns.
- The `Timestamp` column is redundant, as it is the timestamp representation of date and time together. 

In [46]:
# Drop the column 'Unnamed: 16 (all Nulls)
# Drop the 'Timestamp' column (Redundant)
# Drop the 'Latitude' & 'Longitude' columns (Unique)

df_complete.drop(['Latitude', 'Longitude', 'Timestamp', 'Unnamed: 16'], axis=1, inplace = True) 

Let us rename the columns to make the analysis of the data a bit easier:

In [47]:
# rename columns

df_complete.rename(columns = {'Time GMT -4': 'Time',
                    'Ozone - Low Conc.': 'Ozone',
                    'Hydrogen Sulfide - Low Conc.': 'H2S',
                    'Total VOCs (ppm) - PID': 'VOC',
                    'Carbon Dioxide - Low Conc.': 'CO2',
                    'Particulate Matter 1': 'PM1',
                    'Particulate Matter 2.5': 'PM2.5',
                    'Particulate Matter 10': 'PM10',
                    'Temperature (Internal)': 'Temp_int',
                    'Temperature (External)': 'Temp_ext',
                    'Humidity (Internal)': 'Humidity_int',
                    'Humidity (External)': 'Humidity_ext'}, inplace = True)

`Date` and `Time` are stored as strings in the dataset. We concatenate them and convert their type into `datetime`:

In [48]:
# Date & Time

# concatenate Date and Time
datetime = df_complete['Date'] + ' ' + df_complete['Time']

# convert to datetime
datetime = [dt.datetime.strptime(x, '%d/%m/%Y %H:%M:%S') for x in datetime]

# drop the columns 'Date' & 'Time'
df_complete.drop(['Date', 'Time'], axis=1, inplace=True)

# create a column `Time`
df_complete['Time'] = datetime

# set the 'Time' as the index for the dataset
df_complete.set_index('Time', inplace=True)

Now, let's resample the hourly and daily dataset from the data:

In [49]:
# Hourly-based Resample of the data
df_hourly = df_complete.resample('60Min').mean()

In [50]:
# Daily-based Resample of the data 
df_daily = df_hourly.resample('D').mean()

I think it makes sense to round the values of some features, e.g., the temperature and humidity measures:

In [51]:
# Round the values in the Minutly, Hourly, and Daily Datasets

# round internal temperatures 
df_complete['Temp_int'] = [round(x) for x in df_complete['Temp_int']]
df_hourly['Temp_int'] = [round(x) for x in df_hourly['Temp_int']]
df_daily['Temp_int'] = [round(x) for x in df_daily['Temp_int']]

# round the external temperatures
df_complete['Temp_ext'] = [round(x) for x in df_complete['Temp_ext']]
df_hourly['Temp_ext'] = [round(x) for x in df_hourly['Temp_ext']]
df_daily['Temp_ext'] = [round(x) for x in df_daily['Temp_ext']]

# round the internal humidity
df_complete['Humidity_int'] = [round(x) for x in df_complete['Humidity_int']]
df_hourly['Humidity_int'] = [round(x) for x in df_hourly['Humidity_int']]
df_daily['Humidity_int'] = [round(x) for x in df_daily['Humidity_int']]

# round the external humidity
df_complete['Humidity_ext'] = [round(x) for x in df_complete['Humidity_ext']]
df_hourly['Humidity_ext'] = [round(x) for x in df_hourly['Humidity_ext']]
df_daily['Humidity_ext'] = [round(x) for x in df_daily['Humidity_ext']]


# round the Ozone values 
df_complete['Ozone'] = [round(x, 2) for x in df_complete['Ozone']]
df_hourly['Ozone'] = [round(x, 2) for x in df_hourly['Ozone']]
df_daily['Ozone'] = [round(x, 2) for x in df_daily['Ozone']]

# round H2S values
df_complete['H2S'] = [round(x, 5) for x in df_complete['H2S']]
df_hourly['H2S'] = [round(x, 5) for x in df_hourly['H2S']]
df_daily['H2S'] = [round(x, 5) for x in df_daily['H2S']]

# round total VOCs
df_complete['VOC'] = [round(x, 3) for x in df_complete['VOC']]
df_hourly['VOC'] = [round(x, 3) for x in df_hourly['VOC']]
df_daily['VOC'] = [round(x, 3) for x in df_daily['VOC']]

# round CO2
df_complete['CO2'] = [round(x) for x in df_complete['CO2']]
df_hourly['CO2'] = [round(x) for x in df_hourly['CO2']]
df_daily['CO2'] = [round(x) for x in df_daily['CO2']]

# round PM10
df_complete['PM10'] = [round(x, 3) for x in df_complete['PM10']]
df_hourly['PM10'] = [round(x, 3) for x in df_hourly['PM10']]
df_daily['PM10'] = [round(x, 3) for x in df_daily['PM10']]

In [52]:
df_complete.head(3)

,Ozone,H2S,VOC,CO2,PM1,PM2.5,PM10,Temp_int,Humidity_int,Temp_ext,Humidity_ext
Time,,,,,,,,,,,
2020-05-26 08:51:45,0.21,0.000,0.266,467,18.270,19.519,19.759,24,46,23,48
2020-05-26 08:52:44,0.22,0.001,0.268,467,14.326,15.149,15.149,24,46,23,48
2020-05-26 08:53:44,0.22,0.001,0.269,468,10.830,11.453,11.453,24,46,23,48


In [53]:
df_hourly.head(3)

,Ozone,H2S,VOC,CO2,PM1,PM2.5,PM10,Temp_int,Humidity_int,Temp_ext,Humidity_ext
Time,,,,,,,,,,,
2020-05-26 08:00:00,0.21,0.00080,0.268,471,13.714400,14.582200,14.678,24,46,23,48
2020-05-26 09:00:00,0.21,0.00027,0.763,512,5.331483,5.711433,5.800,24,47,23,50
2020-05-26 10:00:00,0.22,0.00017,0.487,553,5.477121,5.907345,6.047,24,49,24,52


In [54]:
df_daily.head(3)

,Ozone,H2S,VOC,CO2,PM1,PM2.5,PM10,Temp_int,Humidity_int,Temp_ext,Humidity_ext
Time,,,,,,,,,,,
2020-05-26,0.21,0.00009,0.340,608,7.657536,8.237709,7.695,24,51,24,51
2020-05-27,0.20,0.00058,0.521,542,8.853346,9.498757,9.658,24,53,25,50
2020-05-28,0.30,0.00002,0.186,399,6.904472,7.452823,7.626,25,62,24,65


In [55]:
df_complete.tail(3)

,Ozone,H2S,VOC,CO2,PM1,PM2.5,PM10,Temp_int,Humidity_int,Temp_ext,Humidity_ext
Time,,,,,,,,,,,
2020-06-09 08:48:25,0.40,0.0,0.180,381,4.432,5.080,5.462,26,39,23,43
2020-06-09 08:49:03,0.43,0.0,0.179,388,6.207,7.307,8.094,26,39,23,44
2020-06-09 08:50:02,0.39,0.0,0.178,381,6.368,7.282,7.815,26,39,23,44


In [56]:
df_hourly.tail(3)

,Ozone,H2S,VOC,CO2,PM1,PM2.5,PM10,Temp_int,Humidity_int,Temp_ext,Humidity_ext
Time,,,,,,,,,,,
2020-06-09 06:00:00,0.31,0.00012,0.186,435,5.859052,6.354155,6.531,26,36,20,54
2020-06-09 07:00:00,0.36,0.00000,0.184,394,6.573183,7.188367,7.457,26,38,21,50
2020-06-09 08:00:00,0.38,0.00000,0.185,391,6.225521,6.950750,7.335,26,38,22,47


In [57]:
df_daily.tail(3)

,Ozone,H2S,VOC,CO2,PM1,PM2.5,PM10,Temp_int,Humidity_int,Temp_ext,Humidity_ext
Time,,,,,,,,,,,
2020-06-07,0.31,0.00000,0.133,338,2.209994,2.472332,2.631,26,34,20,47
2020-06-08,0.35,0.00000,0.139,341,3.119952,3.459822,3.643,26,33,22,41
2020-06-09,0.34,0.00004,0.171,397,5.472919,5.982673,6.199,26,34,20,49


In [58]:
# Serialize the datasets

df_complete.to_pickle('Data/df_minutely')
df_hourly.to_pickle('Data/df_hourly')
df_daily.to_pickle('Data/df_daily')